In [ ]:
import os
import sys
import time
os.environ['TF_ENABLE_ONEDNN_OPTS'] = str(0)

import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
from tensorflow import keras  # tf.keras
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
print("python", sys.version)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

In [ ]:
assert sys.version_info >= (3, 5) # Python ≥3.5 required
assert tf.__version__ >= "2.0"    # TensorFlow ≥2.0 required

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print(gpus)
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            # logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            # print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

mirrored_strategy = None

In [ ]:
from datetime import datetime, timedelta
import sys 
sys.path.append('..')
import json

In [ ]:
from train_params_test import *
from train_utility import *
from trans import *

In [ ]:
def upgrade_file(path):
    with open(path, "+tw") as f:
        f.write("# Sorry, the content is removed.")
        f.write("\n# Please ask Mike for the content.")

In [ ]:
folders = [dir_data, dir_datasets, dir_candles, dir_Checkpoint, dir_CSVLogs]
for folder in folders:
    if not os.path.isdir(folder):
        os.mkdir(folder)

In [ ]:
#==================== Load candle data into 'table' with shape of (time, markets, 10 fields) ====================
Candles = np.load( os.path.join( dir_candles, "table-" + CandleFile + ".npy") )
Candles = np.swapaxes(Candles, 0, 1)
print("Candles: {}".format(Candles.shape))

In [ ]:
market = 5
Show_Price_Volume_10(Candles[:, market, :], 1, 1, 5000)

In [ ]:
Event_Free_Learning_Scheme_10(Candles[:, market, :], 3, 30, 5000)

In [ ]:
#==================== Delete 7 candle fields from 'Candles'. ====================
# Candles.shape becomes (time, markets, ['ClosePrice', 'BaseVolume', 'BuyerBaseVolume'] )

CandleMarks = Candles[:, :, 9] # keep it for later use
Candles = np.delete(Candles, [0, 1, 2, 5, 6, 8, 9], axis = 2) # delete Open, High, Low, qVolume, #Trades, bQVolume, CandleMarks
assert (~np.isfinite(Candles)).any() == False

table_markets = []
with open( os.path.join( dir_candles, "reports-" + CandleFile + ".json"), "r") as f:
    reports = json.loads(f.read())
print(reports[:2])

markets = [ s[0: s.find(':')] for s in reports if 'Success' in s ]
assert Candles.shape[1] == len(markets)
print(Candles.shape, len(markets), markets[:2])

In [ ]:
start_ts, interval_s, timestamps_abs = get_timestamps(CandleFile, Candles)
print(start_ts, interval_s, timestamps_abs.shape, timestamps_abs[:3])

In [ ]:
#==================== Define Data ====================

Data = Candles[:, :, :]   # (time:, all markets, 20 fields)

In [ ]:
#===================== Find marketrank ==================

check = np.array([ np.argmax(Data[:, m, 0]>0) / Data.shape[0] * 100 for m in range(len(markets)) ])
permute = np.argsort(check)
marketrank = [ (markets[m], 100 - round(np.argmax(Data[:, m, 0]>0) / Data.shape[0] * 100)) for m in permute ]
# marketrank = [ markets[m] for m in permute ]

batch = 10
for i in range(0, len(markets), batch):
    print(marketrank[i: i+batch])

In [ ]:
enFields = ['ClosePrice', 'BaseVolume', 'BuyerBaseVolume']

x_indices, y_indices, chosen_markets, chosen_fields = \
get_formation_params(
    enFields, markets, marketrank,
    min_true_candle_percent_x, chosen_fields_x_names, min_true_candle_percent_y, chosen_fields_y_names
)

In [ ]:
len_usdt = len("USDT")
chosen_market_names = [markets[market][:-len_usdt] for market in chosen_markets]
batch = 10
for i in range(0, len(chosen_market_names), batch):
    print(chosen_market_names[i: i+batch])
print(len(chosen_market_names))

In [ ]:
#==================== Generate event-free data into Data ====================
# Data loses heading items.
# Do it before: Permute Data in time

alpha = 3; beta = 3 # beta is used in 'get_eFree_with_plot'. Ugly coupling.
event_free_data_loss = 3 * ( alpha * SmallSigma + LargeSigma)
eFree = np.zeros( (Data.shape[0] - event_free_data_loss, len(chosen_markets), len(chosen_fields)), dtype = np.float32 )

for market in chosen_markets:
    for field in chosen_fields:
        sSigma = SmallSigma
        if enFields[field] == 'BaseVolume': sSigma = SmallSigma * alpha
        P, maP, logP, log_maP, event, eventFree = \
        get_eFree_with_plot(markets[market], enFields[field], Data[:, market, field], sSigma, 
                            LargeSigma, Data.shape[0] - event_free_data_loss, noPlot=eFreeNoPlot, noLog=eFreeNoLog)
        Data[event_free_data_loss:, market, field] = eventFree

Data = Data[event_free_data_loss: ]

print(Data.shape)

In [ ]:
Time = get_time_features(timestamps_abs)
size_time = Time.shape[1]

Time = Time[event_free_data_loss: ]
assert Data.shape[0] == Time.shape[0]
print(Candles.shape, Time.shape)

In [ ]:
Standard = None

if Standardization:
    Data, Standard = standardize(Data, chosen_markets, chosen_fields)

In [ ]:
fig = plt.figure(figsize=(16,3))
ax = fig.add_subplot(111)
ax.set_title("Features are custom-standardized" if Standardization else "Features are not standardized")
for market in chosen_markets:
    for field in chosen_fields:
        ax.plot(Data[:, market, field], label = "{} @ {}".format(enFields[field], markets[market][:-4])) # -4: 'USDT'
ax.legend(loc = 'upper left')
plt.show()

In [ ]:
sample_anchores_t, sample_anchores_v = get_sample_anchores(Data, Nx, Ny, Ns)

print(len(sample_anchores_t), len(sample_anchores_v))

In [ ]:
Dataset_train, Dataset_valid, dx, dy = get_datasets(
    Reuse_files,
    CandleFile, dir_datasets, Data, Time_into_X, Time_into_Y, Time, 
    sample_anchores_t, sample_anchores_v,
    Nx, x_indices, Ny, y_indices, nFiles_t, nFiles_v, n_readers, size_time,
    BatchSize, shuffle_batch, Transformer, nPrefetch
)

In [ ]:
model = None

if mirrored_strategy is None:
    model = build_model(
        dx, dy, Num_Layers, Num_Heads, Factor_FF, repComplexity, Dropout_Rate,
        HuberThreshold, CancleLossWeight, TrendLossWeight
    )
else:
    with mirrored_strategy.scope():
        model = build_model(
            dx, dy, Num_Layers, Num_Heads, Factor_FF, repComplexity, Dropout_Rate,
            HuberThreshold, CancleLossWeight, TrendLossWeight
    )

In [ ]:
callbacks = get_callbacks(
    checkpoint_filepath, Checkpoint_Monitor, 
    csvLogger_filepath, 
    EarlyStopping_Min_Monitor, EarlyStopping_Patience
)

try:
    model.load_weights(checkpoint_filepath)
except:
    print("Failed to load a checkpoint")
    pass

In [ ]:
model.fit(
    Dataset_train, # x and y_true
    validation_data=Dataset_valid, 
    epochs=20, #Epochs_Initial,
    callbacks=callbacks
)

In [ ]:
columns = ('loss', 'val_loss', 'trans_mTA', 'val_trans_mTA', 'trans_1_loss', 'val_trans_1_loss')
plot_csv_log_history(csvLogger_filepath, columns)